In [62]:
import pinocchio
import numpy as np 
import time 
import pymanopt 
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem

from utils import *
import networkx as nx
from scipy import optimize
from scipy.optimize import Bounds

import tqdm

LOWER = "lower_limit"
UPPER = "upper_limit"
BOUNDED = "bounded"
BELOW = "below"
ABOVE = "above"
TYPE = "type"
OBSTACLE = "obstacle"
ROBOT = "robot"
END_EFFECTOR = "end_effector"
RADIUS = "radius"
DIST = "weight"
POS = "pos"
BASE = "base"
ROOT = None
ANCHOR = "anchor"
BASE_GRAPH = "base_graph"
UNDEFINED = None


deg2rad = np.pi/180.
rad2deg = 180./np.pi

scale = 1

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
def forwardKinematics(model,data,q,scale=1):
    pinocchio.forwardKinematics(model,data,q)
    for idx, oMi in enumerate(data.oMi):
        oMi.translation = scale*oMi.translation


In [64]:
def create_base_graph(model,data,axis_length):

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))

    q_init = pinocchio.neutral(model)
    pinocchio.forwardKinematics(model,data,q_init)
    
    robot_name = model.names[1]
    ROOT = robot_name

    base = nx.empty_graph()

    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            cur_pos, aux_cur_pos = (
                oMi.translation*scale,
                pinocchio.SE3.act(pinocchio.SE3(oMi.rotation,oMi.translation*scale),trans_z).translation,
            )
            #print(name,cur_pos,aux_cur_pos)
            dist = np.linalg.norm(cur_pos - aux_cur_pos)

            # Add nodes for joint and edge between them
            base.add_nodes_from(
                [(cur, {POS: cur_pos}), (aux_cur, {POS: aux_cur_pos})]
            )
            base.add_edge(
                cur, aux_cur, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False}
            )

            # If there exists a preceeding joint, connect it to new
            if idx != 0:
                pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')
                for u in [pred, aux_pred]:
                    for v in [cur, aux_cur]:
                        dist = np.linalg.norm(
                            base.nodes[u][POS] - base.nodes[v][POS]
                        )
                        base.add_edge(
                            u,
                            v,
                            **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False},
                        )
        elif idx == 1:
            #print(name,oMi.translation)
            base = nx.DiGraph(
                [
                    (robot_name, "x"),
                    (robot_name, "y"),
                    (robot_name, robot_name+'_tilde'),
                    ("x", "y"),
                    ("y", robot_name+'_tilde'),
                    (robot_name+'_tilde' ,"x"),
                ]
            )
            base.add_nodes_from(
                [
                    ("x", {POS: np.array([axis_length, 0, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    ("y", {POS: np.array([0, -axis_length, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    (robot_name, {POS: oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                    (robot_name+'_tilde', {POS: np.array([0, 0, axis_length]) + oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                ]
            )
            for u, v in base.edges():
                base[u][v][DIST] = np.linalg.norm(base.nodes[u][POS] - base.nodes[v][POS])
                base[u][v][LOWER] = base[u][v][DIST]
                base[u][v][UPPER] = base[u][v][DIST]
                base[u][v][ANCHOR] = True
                base[u][v][BOUNDED] = []

    # Set node type to robot
    nx.set_node_attributes(base, [ROBOT], TYPE)
    base.nodes[ROOT][TYPE] = [ROBOT, BASE]
    base.nodes[ROOT + '_tilde'][TYPE] = [ROBOT, BASE]

    return base

In [65]:

def norm(x):
    return x/max(np.linalg.norm(x),10**-9)

def goal_graph(model, data, G, axis_length, position, direction = None, anchor = True):
    
    end_effector = [model.names[-1]]
    if direction is not None:
        end_effector.append(model.names[-1]+'_tilde')

    list_nodes_base = ['x','y',model.names[1],model.names[1]+'_tilde']

    for i, cur_end_effector in enumerate(end_effector):
        pos_end = position

        if direction is not None and i == 1:
            pos_end = pos_end + direction*axis_length
        
        G.nodes[cur_end_effector][POS] = pos_end

        for cur_base in list_nodes_base:
            
            if not((i == 1) and (len(end_effector)==1)):
                dist = np.linalg.norm(pos_end-G.nodes[cur_base][POS])

                G.add_edge(
                            cur_base, cur_end_effector, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: anchor}
                        )
    
    if anchor:

        list_edges_anchor = []
        list_other_edges = []
        new_edge_order  = []
        A_anchor = []

        for key ,anch_ in nx.get_edge_attributes(G,ANCHOR).items():

            if anch_:
                list_edges_anchor.append(key)
                A_anchor.append(norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
                #print(key,' : ',norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
            else:
                list_other_edges.append(key)
        new_edge_order = list_edges_anchor.copy() #.extend(list_other_edges)

        for e in list_other_edges:
            new_edge_order.append(e)

        A_anchor = np.array(A_anchor).transpose()
        
    else:

        list_edges_anchor = []
        list_other_edges = [e for e in G.edges()]

        new_edge_order = list_other_edges
        A_anchor = []

    n = len(list_other_edges)
    n_anchor = len(list_edges_anchor)

    weight = nx.get_edge_attributes(G, DIST)
    D = np.diag([weight[i] for i in new_edge_order])

    C = incidence_matrix_(G,oriented=True, edgelist=new_edge_order).toarray()
    C = np.array(C)

    Y_init = np.zeros((3,n))

    for i, e in enumerate(list_other_edges):
        i = i 
        p1 = G.nodes[e[0]][POS]
        p2 = G.nodes[e[1]][POS]
        Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

    if anchor:
        Y_init = [A_anchor,Y_init]
    return G, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init

In [189]:
def simple_IK(d ,C , C_joints_limit ,D ,D_joints ,n_below, n_above, Anchor_ ,M ,max_iter,ind=None ,W=None ,Y_init=None ,use_rand=False):
    if  W is None:
        W = np.eye(D.shape[0])

    n_anchor = Anchor_.shape[1]
    Anchor = np.zeros((d,n_anchor))
    Anchor[:3,:] = Anchor_


    M = manifolds.Product([manifolds.ConstantFactory(Anchor),
            manifolds.Oblique(d,D.shape[1]-Anchor.shape[1])])

    Q = np.zeros((C_joints_limit.shape[1],C_joints_limit.shape[1]))
    Q_ = np.zeros((C_joints_limit.shape[1],C_joints_limit.shape[1]))

    Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D
    Q1 = D@W@D

    Q[:Q2.shape[0],:Q2.shape[1]] = Q2
    Q_[:Q1.shape[0],:Q1.shape[1]] = Q1

    Y = M.random_point()
    ind = [0]
    ind.extend([y.shape[1] for y in Y])
    ind = np.array(ind)
    ind = np.cumsum(ind)
    n = len(Y)
    
    @pymanopt.function.numpy(M)
    def cost(*Y):
        Y = np.concatenate(Y,axis=1)
        print(Y.shape,Q_.shape,Q.shape)
        return np.trace(Q_ + Q@Y.transpose()@Y) 

    @pymanopt.function.numpy(M)
    def euclidean_gradient(*Y):
        Y = np.concatenate(Y,axis=1)
        grad = 2*Q@Y.transpose()
        return [grad.transpose()[:,start:end] for start, end in zip(ind[:-1],ind[1:])]
                            
    @pymanopt.function.numpy(M)
    def euclidean_hessian(*U):
        #[print(u.shape) for u in U[n:]]
        U = np.concatenate(U[n:],axis=1)
        hess = 2*Q@U.transpose()
        return [hess.transpose()[:,start:end] for start, end in zip(ind[:-1],ind[1:])]


    problem = Problem(manifold=M, 
                        cost=cost,
                        euclidean_gradient=euclidean_gradient, 
                        euclidean_hessian=euclidean_hessian,
                        )
    #grad = euclidean_gradient(Y)
    #print([y.shape for y in grad])
    optimizer = optimizers.TrustRegions(max_iterations=max_iter,use_rand=use_rand,min_gradient_norm=10**-12,verbosity=2)

    if  Y_init is None:
        Y_init_ = M.random_point()
    elif n_anchor > 1:
        if Y_init[0].shape[0] == d:
            Y_init_ = Y_init
        elif Y_init[0].shape[0] < d:
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init[0].shape[0],:] = Y_init[1]
            Y_init_ = [Anchor, Y_init_]
        else:
            if len(Y_init) == 2:
                U,S,V = np.linalg.svd(Y_init[1],full_matrices=False)
            else:
                U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = [Anchor, U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)]


    Y_star = optimizer.run(problem,initial_point=Y_init_).point

    return Y_star


In [180]:
W = np.diag(1./np.diag(D)**2)
M=None
n_below = len(list_edges_below)
n_above = len(list_edges_above)
#Y_D5 = simple_IK(5,C,D,A_anchor,M,max_iter=100,ind=None,W=None,Y_init=None,use_rand=False)
#Y_D4 = simple_IK(4,C,D,A_anchor,M,max_iter=2000,ind=None,W=None,Y_init=None,use_rand=False)
Y_D3 = simple_IK(3,C,C_joints_limit,D,np.array(D_joints_limit),n_below ,n_above ,A_anchor,M,max_iter=2000,ind=None,W=None,Y_init=None,use_rand=False)

(3, 57) (57, 57) (57, 57)
Optimizing...
                                            f: +8.029792e+01   |grad|: 1.804545e+01
(3, 57) (57, 57) (57, 57)
acc TR+   k:     1     num_inner:     0     f: +4.085684e+01   |grad|: 1.261636e+01   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc TR+   k:     2     num_inner:     0     f: +3.825571e+00   |grad|: 2.093278e+00   exceeded trust region
(3, 57) (57, 57) (57, 57)
REJ TR-   k:     3     num_inner:     2     f: +3.825571e+00   |grad|: 2.093278e+00   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc TR+   k:     4     num_inner:     2     f: +2.065478e+00   |grad|: 1.257705e+00   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     5     num_inner:     2     f: +1.003747e+00   |grad|: 1.168046e+00   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     6     num_inner:     4     f: +5.508830e-01   |grad|: 1.179006e+00   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     7     num_inner:     5     f:

In [67]:
def direction_to_position(Y,C,D,data):
    #print(Y.shape, C.shape, D.shape)
    X = np.linalg.pinv(C@C.transpose())@C@D@Y.transpose()
    #print(X)
    X = X - X[0,:] + data.oMi[1].translation*scale
    
    #if X[0,0]<0:
      #  X = -X
    
    v = X[1,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    v = X[2,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/np.linalg.norm(e2))
    X = (X - X[0,:])@R.transpose() + X[0,:]
    flip = 1

    if X[3,2]<axis_length:
        flip = -1
    v = X[2,:] - X[0,:]
    R = np.diag((1,1,flip))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    return X

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [111]:
def skew(x):
    """
    Creates a skew symmetric matrix from vector x
    """
    X = np.array([[0, -x[2], x[1]], [x[2], 0, -x[0]], [-x[1], x[0], 0]])
    return X


def joint_variables(model,data, G, T_final, axis_length):
    """
    Finds the set of decision variables corresponding to the
    graph realization G.

    :param G: networkx.DiGraph with known vertex positions
    :param T_final: poses of end-effectors in case two final frames aligned along z
    :returns: Dictionary of joint angles
    :rtype: Dict[str, float]
    """
    tol = 1e-10
    
    T = {}
    T[model.names[1]] = data.oMi[1] #G.nodes[model.names[1]][POS]

    pinocchio.forwardKinematics(model,data,pinocchio.neutral(model))
    T_zero = {name: oMi for name, oMi in zip(model.names, data.oMi)}

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))
    
    # resolve scale
    x_hat = G.nodes["x"][POS] - G.nodes[model.names[1]][POS]
    y_hat = G.nodes["y"][POS] - G.nodes[model.names[1]][POS]
    z_hat = G.nodes[model.names[1]+'_tilde'][POS] - G.nodes[model.names[1]][POS]

    # resolve rotation and translation
    x = norm(x_hat)
    y = norm(y_hat)
    z = norm(z_hat)
    R = np.vstack((x, -y, z)).T
    B = pinocchio.SE3(R, np.zeros((3,))) #G.nodes[ROOT][POS])

    omega_z = skew(np.array([0,0,1]))

    theta = {}
    
    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')

            T_prev = T[pred]
            
            T_prev_0 = T_zero[pred] # previous p xf at 0
            T_0 = T_zero[cur] # cur p xf at 0
            T_0_q = pinocchio.SE3.act(T_zero[cur],trans_z) # cur q xf at 0
            T_rel = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0) # relative xf
            ps_0 = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0).translation # relative xf
            qs_0 = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0_q).translation # rel q xf

            # predicted p and q expressed in previous frame
            p = pinocchio.SE3.act(pinocchio.SE3.inverse(B),G.nodes[cur][POS])
            qnorm = G.nodes[cur][POS] + (
                G.nodes[aux_cur][POS] - G.nodes[cur][POS]
            ) / np.linalg.norm(G.nodes[aux_cur][POS] - G.nodes[cur][POS])
            q = pinocchio.SE3.act(pinocchio.SE3.inverse(B),qnorm)
            #print(T_prev)
            ps = pinocchio.SE3.inverse(T_prev).rotation.dot(p - T_prev.translation)  # in prev. joint frame
            qs = pinocchio.SE3.inverse(T_prev).rotation.dot(q - T_prev.translation)  # in prev. joint frame

            theta[cur] = np.arctan2(-qs_0.dot(omega_z).dot(qs), qs_0.dot(omega_z.dot(omega_z.T)).dot(qs))
            rot_axis_z = pinocchio.SE3(Rot_z(theta[cur]),np.zeros((3,)))
            T[cur] = pinocchio.SE3.act(pinocchio.SE3.act(T_prev,rot_axis_z),T_rel)

    # if the rotation axis of final joint is aligned with ee frame z axis,
    # get angle from EE pose if available
    #if ((T_final is not None) and (la.norm(cross(T_rel.trans, np.asarray([0, 0, 1]))) < tol)):
     #   T_th = (T[cur]).inv().dot(T_final[ee]).as_matrix()
     #   theta[ee] = wraptopi(theta[ee] +  arctan2(T_th[1, 0], T_th[0, 0]))

    return theta

In [127]:
def root_angle_limits(G,model,data,axis_length):
    upper_limits = np.minimum(-model.lowerPositionLimit,model.upperPositionLimit)
    limited_joints = [] 
    i = 2
    T1 = data.oMi[1]
    base_names = ["x", "y"]
    names = [model.names[i],model.names[i]+'_tilde']
    T_axis = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))

    for base_node in base_names:
        for node in names:
            T0 = pinocchio.SE3.Identity()
            T0.translation = G.nodes[base_node][POS]
            if node == model.names[i]:
                T2 = data.oMi[i]
            else:
                T2 = pinocchio.SE3.act(data.oMi[i],T_axis)

            N = T1.rotation[0:3, 2]
            C = T1.translation + (N.dot(T2.translation - T1.translation)) * N
            r = np.linalg.norm(T2.translation - C)
            P = T0.translation
            d_max, d_min = max_min_distance_revolute(r, P, C, N)
            d = np.linalg.norm(T2.translation - T0.translation)

            if d_max == d_min:
                limit = False
            elif d == d_max:
                limit = BELOW
            elif d == d_min:
                limit = ABOVE
            else:
                limit = None

            if limit:
                T_rel = pinocchio.SE3.act(pinocchio.SE3.inverse(T1),data.oMi[i])
                if node != model.names[i]:
                    T_rel = pinocchio.SE3.act(T_rel,T_axis)

                d_limit = np.linalg.norm(
                    pinocchio.SE3.act(pinocchio.SE3.act(T1,pinocchio.SE3(Rot_z(upper_limits[0]),np.zeros((3,)))),T_rel).translation
                    - T0.translation
                )

                if limit == ABOVE:
                    d_max = d_limit
                else:
                    d_min = d_limit
                limited_joints += [model.names[i]]  # joint at p0 is limited
            
            G.add_edge(base_node, node)
            if d_max == d_min:
                G[base_node][node][DIST] = d_max
            G[base_node][node][BOUNDED] = [limit]
            G[base_node][node][UPPER] = d_max
            G[base_node][node][LOWER] = d_min

    return G

In [70]:
def set_limits(G,model,data,axis_length):
    """
    Sets known bounds on the distances between joints.
    This is induced by link length and joint limits.
    """
    T_axis = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))
    upper_limits = np.minimum(-model.lowerPositionLimit,model.upperPositionLimit)

    limited_joints = []  # joint limits that can be enforced


    pinocchio.forwardKinematics(model,data,pinocchio.neutral(model))
    T_zero = {name: oMi for name, oMi in zip(model.names, data.oMi)}
    
    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 2:
            cur, prev = name, model.names[idx - 2]
            names = [
                (prev, name),
                (prev,name+'_tilde'),
                (prev+'_tilde', name),
                (prev+'_tilde', name+'_tilde'),
            ]
            for ids in names:
                T0, T1, T2 = [
                    T_zero[model.names[idx-2]],
                    T_zero[model.names[idx-1]],
                    T_zero[model.names[idx]],
                ]

                if '_tilde' in ids[0]:
                    T0 = pinocchio.SE3.act(T0,T_axis)
                if '_tilde' in ids[1]:
                    T2 = pinocchio.SE3.act(T2,T_axis)

                N = T1.rotation[0:3, 2]
                C = T1.translation + (N.dot(T2.translation - T1.translation)) * N
                r = np.linalg.norm(T2.translation - C)
                P = T0.translation
                d_max, d_min = max_min_distance_revolute(r, P, C, N)

                d = np.linalg.norm(T2.translation - T0.translation)
                if d_max == d_min:
                    limit = False
                elif d == d_max:
                    limit = BELOW
                elif d == d_min:
                    limit = ABOVE
                else:
                    limit = None

                if limit:

                    rot_limit = pinocchio.SE3(Rot_z(upper_limits[idx-1]),np.zeros((3,)))

                    T_rel = pinocchio.SE3.act(pinocchio.SE3.inverse(T1),T2)

                    d_limit = np.linalg.norm(pinocchio.SE3.act(pinocchio.SE3.act(T1,rot_limit),T_rel).translation - T0.translation)

                    if limit == ABOVE:
                        d_max = d_limit
                    else:
                        d_min = d_limit

                    limited_joints += [cur]

                G.add_edge(ids[0], ids[1])
                if d_max == d_min:
                    G[ids[0]][ids[1]][DIST] = d_max
                G[ids[0]][ids[1]][BOUNDED] = [limit]
                G[ids[0]][ids[1]][UPPER] = d_max
                G[ids[0]][ids[1]][LOWER] = d_min
    return G, limited_joints

In [71]:
def constraints_graph(model, data, G, D, new_edge_order, axis_length):
    D_tilde = {name: [val, ind] for ind, (name, val) in enumerate(zip(new_edge_order,D))}

    list_edges_below = []
    list_edges_above = []

    D_below = []
    D_above = []

    path = {}

    for n1 in G.nodes():
        for n2 in G.nodes():

            path[(n1,n2)] = []

            if n1 != n2 and ((n1,n2) not in new_edge_order):
                for p in nx.all_simple_edge_paths(G, n1, n2, 2):
                    if len(p)>1:
                        path[(n1,n2)].append(p)

    G = root_angle_limits(G,model,data,axis_length)
    G, lim_joints = set_limits(G,model,data,axis_length)

    for edge, val in nx.get_edge_attributes(G,BOUNDED).items():

        if BELOW in val:
            D_below.append(G[edge[0]][edge[1]][LOWER])
            list_edges_below.append(edge)
        if ABOVE in val:
            D_above.append(G[edge[0]][edge[1]][UPPER])
            list_edges_above.append(edge)

    list_edges_joints_limit = list_edges_below.copy()
    list_edges_joints_limit.extend(list_edges_above)
    D_joints_limit = D_below.copy()
    D_joints_limit.extend(D_above)
    
    C_joints_limit = np.zeros((len(list_edges_joints_limit),len(list_edges_joints_limit)+len(new_edge_order)))
    
    for idx, e in enumerate(list_edges_joints_limit):
        p = path[e][0]
        
        C_joints_limit[idx,D_tilde[p[0]][1]] = D_tilde[p[0]][0]
        C_joints_limit[idx,D_tilde[p[1]][1]] = D_tilde[p[1]][0]

        C_joints_limit[idx,len(new_edge_order)+idx] = -1
    
    return G, D_joints_limit, list_edges_joints_limit, C_joints_limit, list_edges_below, list_edges_above


In [192]:
G1, D_joints_limit, list_edges_joints_limit, C_joints_limit, list_edges_below, list_edges_above = constraints_graph(model, data, G1, np.diag(D), new_edge_order, axis_length)

In [142]:
print(C_joints_limit.shape,len(list_edges_below),len(list_edges_above))
nx.get_edge_attributes(G1,BOUNDED)

(13, 57) 7 6


{('lbr_iiwa_joint_1', 'x'): [],
 ('lbr_iiwa_joint_1', 'y'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_1_tilde'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_2'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_2_tilde'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7_tilde'): [],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_3'): ['below'],
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_3_tilde'): ['below'],
 ('x', 'y'): [],
 ('x', 'lbr_iiwa_joint_7'): [],
 ('x', 'lbr_iiwa_joint_7_tilde'): [],
 ('x', 'lbr_iiwa_joint_2'): [False],
 ('x', 'lbr_iiwa_joint_2_tilde'): [None],
 ('y', 'lbr_iiwa_joint_1_tilde'): [],
 ('y', 'lbr_iiwa_joint_7'): [],
 ('y', 'lbr_iiwa_joint_7_tilde'): [],
 ('y', 'lbr_iiwa_joint_2'): [False],
 ('y', 'lbr_iiwa_joint_2_tilde'): [None],
 ('lbr_iiwa_joint_1_tilde', 'x'): [],
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_2'): [],
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_2_tilde'): [],
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7'): [],
 ('lbr_iiwa_j

In [191]:
axis_length = 0.3*scale
urdf_filename="GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
# 'GraphIK/graphik/robots/urdfs/panda_arm.urdf'

model    = pinocchio.buildModelFromUrdf(urdf_filename)
data     = model.createData()

G = create_base_graph(model,data,axis_length)

q0 = np.array([30,35,10,-40,90,0,0])*np.pi/180
#q0 = np.array([0,0,0,0,0,0,0])*np.pi/180
#q0 = pinocchio.randomConfiguration(model)
pinocchio.framesForwardKinematics(model,data,q0)
position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
direction = data.oMi[-1].rotation@np.array([0,0,1])

print(position, direction)
ROOT = model.names[1]

G1, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init_ = goal_graph(model, data, G, axis_length, 
                                                                                                 position, direction = direction, anchor = True)



[0.58081515 0.39732814 0.83122999] [0.77377923 0.57562804 0.26442024]


In [198]:
W = np.diag(1./np.diag(D)**2)
M=None

n_below = len(list_edges_below)
n_above = len(list_edges_above)
#Y_D5 = simple_IK(5,C,D,A_anchor,M,max_iter=100,ind=None,W=None,Y_init=None,use_rand=False)
#Y_D4 = simple_IK(4,C,D,A_anchor,M,max_iter=2000,ind=None,W=None,Y_init=None,use_rand=False)
Y_D3 = simple_IK(3,C,C_joints_limit,D,np.array(D_joints_limit),n_below ,n_above ,A_anchor,M,max_iter=2000,ind=None,W=None,Y_init=None,use_rand=False)

(3, 57) (57, 57) (57, 57)
Optimizing...
                                            f: +2.608632e+00   |grad|: 6.715883e-01
(3, 57) (57, 57) (57, 57)
acc TR+   k:     1     num_inner:     0     f: +1.350107e+00   |grad|: 5.129214e-01   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     2     num_inner:     0     f: +4.644777e-01   |grad|: 3.978447e-01   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     3     num_inner:     2     f: +2.435572e-01   |grad|: 2.854255e-01   exceeded trust region
(3, 57) (57, 57) (57, 57)
acc       k:     4     num_inner:     7     f: +9.202963e-02   |grad|: 1.537539e-01   exceeded trust region
(3, 57) (57, 57) (57, 57)
REJ TR-   k:     5     num_inner:     7     f: +9.202963e-02   |grad|: 1.537539e-01   negative curvature
(3, 57) (57, 57) (57, 57)
acc TR+   k:     6     num_inner:     3     f: +1.559528e-02   |grad|: 3.541801e-02   exceeded trust region
(3, 57) (57, 57) (57, 57)
REJ TR-   k:     7     num_inner:    25     f: +1

In [199]:
Y_star = Y_D3[:2]
e1 = np.array([1,0,0])
e2 = np.array([0,1,0])
e3 = np.array([0,0,1])

if len(Y_star) == 2:
    X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D,data)
else:
    X = direction_to_position(Y_star,C,D,data)

print(X)
print(np.linalg.norm(X[-2,:]-position))
print(np.linalg.norm((X[-1,:]-X[-2,:])/axis_length-direction))
print(position)

for idx,node in enumerate(G1.nodes()):
    G1.nodes[node][POS] = X[idx,:]

theta = joint_variables(model,data, G1, data.oMi[-1], axis_length)
theta = np.array([theta_ for key, theta_ in theta.items()])
pinocchio.forwardKinematics(model,data,np.concatenate([theta,np.zeros((1,))]))
print(np.linalg.norm(position-data.oMi[-1].translation))
print(np.linalg.norm(direction-data.oMi[-1].rotation@np.array([0,0,1])))
print(model.upperPositionLimit*rad2deg)
print(q0*rad2deg)
(theta*rad2deg).tolist()

[[ 0.00000000e+00  0.00000000e+00  1.57500000e-01]
 [ 2.99995629e-01  1.10253598e-06  1.57500000e-01]
 [-4.65640404e-23 -3.00003269e-01  1.57500000e-01]
 [-8.87151231e-06  7.67763038e-06  4.57492360e-01]
 [-2.20531907e-03 -7.84544708e-04  3.60023129e-01]
 [-1.04753986e-01  2.81131487e-01  3.59994168e-01]
 [ 1.19353263e-01  4.34181260e-02  5.18486223e-01]
 [ 2.90567570e-01  1.05868810e-01  7.56825604e-01]
 [ 2.41901425e-01  8.73670352e-02  6.90291550e-01]
 [ 2.58857477e-02  2.64998383e-01  7.98866603e-01]
 [ 3.68999497e-01  2.11746231e-01  7.39861071e-01]
 [ 5.76957608e-01  4.04979975e-01  8.37044959e-01]
 [ 5.17760583e-01  3.51011227e-01  8.10118525e-01]
 [ 5.58116377e-01  1.83924020e-01  1.05597205e+00]
 [ 5.80782910e-01  3.97347282e-01  8.31226518e-01]
 [ 8.12910008e-01  5.70045406e-01  9.10545483e-01]]
3.765335595093434e-05
4.5863058511100985e-05
[0.58081515 0.39732814 0.83122999]
0.0079037299622913
0.011433078372999794
[170. 120. 170. 120. 170. 120. 175.]
[ 30.  35.  10. -40.  90. 

[20.123085260730523,
 37.62413545006357,
 -143.5357666350511,
 38.54219114995794,
 -82.44764861982058,
 -6.913142543617022]

In [181]:
count_success = 0
success_range = 0

fail_range = []
err_pos = []
err_or = []
scale = 1

axis_length = 0.3*scale
urdf_filename="GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
# 'GraphIK/graphik/robots/urdfs/panda_arm.urdf'

model    = pinocchio.buildModelFromUrdf(urdf_filename)
data     = model.createData()

G = create_base_graph(model,data,axis_length)

for i in tqdm.tqdm(range(1)):
    q0 = pinocchio.randomConfiguration(model)
    pinocchio.framesForwardKinematics(model,data,q0)
    position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
    direction = data.oMi[-1].rotation@np.array([0,0,1])

    #print(position, direction)

    G1, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init_ = goal_graph(model, data, G, axis_length, 
                                                                                                 position, direction = direction, anchor = True)
    G1, D_joints_limit, list_edges_joints_limit, C_joints_limit, list_edges_below, ...
    list_edges_above = constraints_graph(model, data, G1, np.diag(D), new_edge_order, axis_length)

    W = np.diag(1./np.diag(D)**2)#np.diag(np.concatenate([np.ones(n_anchor),np.ones(n)]))
    M=None
    Y_D3 = simple_IK(3,C,C_joints_limit,D,np.array(D_joints_limit),A_anchor,M,max_iter=500,ind=None,W=None,Y_init=None,use_rand=False)

        
    Y_star = Y_D3[:2]

    if len(Y_star) == 2:
        #print(ANCHOR)
        X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D,data)
    else:
        X = direction_to_position(Y_star,C,D,data)
    
        
    for idx,node in enumerate(G1.nodes()):
        G1 .nodes[node][POS] = X[idx,:]

    theta = joint_variables(model,data, G1, data.oMi[-1], axis_length)
    theta = np.array([theta_ for key, theta_ in theta.items()])
    pinocchio.forwardKinematics(model,data,np.concatenate([theta,np.zeros((1,))]))
    err_pos.append(np.linalg.norm(position-data.oMi[-1].translation))
    err_or.append(np.linalg.norm(direction-data.oMi[-1].rotation@np.array([0,0,1])))

    if not (False in (theta>model.lowerPositionLimit[:6]).tolist() or False in (theta<model.upperPositionLimit[:6]).tolist()) :
        success_range = success_range + 1
    else:
        fail_range.append([theta,position,direction,q0,X])


  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: simple_IK() missing 2 required positional arguments: 'Anchor_' and 'M'

In [132]:
print(max(err_pos),max(err_or))
#sum(np.array(err_or)<10**-5)
print(theta*rad2deg)
model.upperPositionLimit*rad2deg

1.888583349675042e-05 1.8752470800106118e-05
[ -80.93073625  -97.49998528  153.40061731 -102.29913452 -177.86981199
   27.83645316]


array([170., 120., 170., 120., 170., 120., 175.])